In [1]:
import numpy as np
import pandas as pd
import math
import h5py

import matplotlib.pyplot as plt

import torch

#from itertools import product
from pynwb import NWBHDF5IO
from nlb_tools.nwb_interface import NWBDataset

import warnings
warnings.filterwarnings("ignore")

In [2]:
file = h5py.File('data/000070/sub-Jenkins/sub-Jenkins_ses-20090918_behavior+ecephys.nwb')
io = NWBHDF5IO(file=file, mode='r', load_namespaces=True)
nwb = io.read()
nwb

,x,y,z,imp,location,filtering,group,group_name,gain,offset
id,,,,,,,,,,
1,NaN,NaN,NaN,-1.0,"Pre-Motor Cortex, dorsal",1000Hz,"1 pynwb.ecephys.ElectrodeGroup at 0x11451658384\nFields:\n description: array corresponding to device implanted at PMd\n device: Utah Array(PMd) pynwb.device.Device at 0x11459867024\nFields:\n description: 96 channel utah array\n manufacturer: BlackRock Microsystems\n\n location: Caudal, dorsal Pre-motor cortex, Left hemisphere\n",1,1.0,0.0
2,NaN,NaN,NaN,-1.0,"Pre-Motor Cortex, dorsal",1000Hz,"1 pynwb.ecephys.ElectrodeGroup at 0x11451658384\nFields:\n description: array corresponding to device implanted at PMd\n device: Utah Array(PMd) pynwb.device.Device at 0x11459867024\nFields:\n description: 96 channel utah array\n manufacturer: BlackRock Microsystems\n\n location: Caudal, dorsal Pre-motor cortex, Left hemisphere\n",1,1.0,0.0
3,NaN,NaN,NaN,-1.0,"Pre-Motor Cortex, dorsal",1000Hz,"1 pynwb.ecephys.ElectrodeGroup at 0x11451658384\nFields:\n description: array corresponding to device implanted at PMd\n device: Utah Array(PMd) pynwb.device.Device at 0x11459867024\nFields:\n description: 96 channel utah array\n manufacturer: BlackRock Microsystems\n\n location: Caudal, dorsal Pre-motor cortex, Left hemisphere\n",1,1.0,0.0
4,NaN,NaN,NaN,-1.0,"Pre-Motor Cortex, dorsal",1000Hz,"1 pynwb.ecephys.ElectrodeGroup at 0x11451658384\nFields:\n description: array corresponding to device implanted at PMd\n device: Utah Array(PMd) pynwb.device.Device at 0x11459867024\nFields:\n description: 96 channel utah array\n manufacturer: BlackRock Microsystems\n\n location: Caudal, dorsal Pre-motor cortex, Left hemisphere\n",1,1.0,0.0
,x,y,z,imp,location,filtering,group,group_name,gain,offset
id,,,,,,,,,,
1,NaN,NaN,NaN,-1.0,"Pre-Motor Cortex, dorsal",1000Hz,"1 pynwb.ecephys.ElectrodeGroup at 0x11451658384\nFields:\n description: array corresponding to device implanted at PMd\n device: Utah Array(PMd) pynwb.device.Device at 0x11459867024\nFields:\n description: 96 channel utah array\n manufacturer: BlackRock Microsystems\n\n location: Caudal, dorsal Pre-motor cortex, Left hemisphere\n",1,1.0,0.0
2,NaN,NaN,NaN,-1.0,"Pre-Motor Cortex, dorsal",1000Hz,"1 pynwb.ecephys.ElectrodeGroup at 0x11451658384\nFields:\n description: array corresponding to device implanted at PMd\n device: Utah Array(PMd) pynwb.device.Device at 0x11459867024\nFields:\n description: 96 channel utah array\n manufacturer: BlackRock Microsystems\n\n location: Caudal, dorsal Pre-motor cortex, Left hemisphere\n",1,1.0,0.0
3,NaN,NaN,NaN,-1.0,"Pre-Motor Cortex, dorsal",1000Hz,"1 pynwb.ecephys.ElectrodeGroup at 0x11451658384\nFields:\n description: array corresponding to device implanted at PMd\n device: Utah Array(PMd) pynwb.device.Device at 0x11459867024\nFields:\n description: 96 channel utah array\n manufacturer: BlackRock Microsystems\n\n location: Caudal, dorsal Pre-motor cortex, Left hemisphere\n",1,1.0,0.0


In [3]:
trials_df = nwb.trials.to_dataframe()
trials_df

,start_time,stop_time,target_presentation_time,go_cue_time,reaction_time,move_begins_time,move_ends_time,discard_trial,task_success,trial_type,...,correct_reach,maze_num_targets,maze_num_barriers,novel_maze,target_positions,frame_details,hit_target_position,target_size,barrier_info,timeseries
id,,,,,,,,,,,,,,,,,,,,,
0,4.707200,8.567133,5.570200,6.517200,0.335,6.819200,7.234200,0,1,0,...,0,1,2,1,"[[-130, -57]]","[[[-200]], [[200]], [[-155]], [[155]], [[20]]]","[-130, -57]",13,"[[-16.0, 96.0, 59.0, 22.0], [-62.0, -112.0, 43...","[(0, 3226, Eye pynwb.behavior.SpatialSeries at..."
1,9.407200,12.112167,10.172200,10.202200,0.305,10.479200,10.789200,0,1,0,...,0,1,2,1,"[[120, 70]]","[[[-200]], [[200]], [[-155]], [[155]], [[20]]]","[120, 70]",13,"[[-49.0, -59.0, 22.0, 53.0], [-33.0, -100.0, 5...","[(3226, 2081, Eye pynwb.behavior.SpatialSeries..."
2,18.437233,21.972267,19.209233,19.857233,0.300,20.124233,20.584233,0,1,17,...,1,1,9,0,"[[-77, 82]]","[[[-200]], [[200]], [[-155]], [[155]], [[20]]]","[-77, 82]",13,"[[-33.0, 47.0, 6.0, 37.0], [-77.0, 48.0, 11.0,...","[(5307, 2846, Eye pynwb.behavior.SpatialSeries..."
3,22.777233,26.007267,23.495233,23.959233,0.305,24.229233,24.644233,0,1,8,...,1,1,9,0,"[[132, 56]]","[[[-200]], [[200]], [[-155]], [[155]], [[20]]]","[132, 56]",13,"[[-82.0, -43.0, 8.0, 118.0], [145.0, -105.0, 1...","[(8153, 2566, Eye pynwb.behavior.SpatialSeries..."
4,26.937267,30.317267,27.630267,28.211267,0.300,28.479267,28.979267,0,1,29,...,1,1,9,0,"[[2, 82]]","[[[-200]], [[200]], [[-155]], [[155]], [[20]]]","[2, 82]",13,"[[-9.0, 52.0, 8.0, 43.0], [-50.0, 91.0, 64.0, ...","[(10719, 2741, Eye pynwb.behavior.SpatialSerie..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2631,15586.530559,15590.185559,15587.318559,15587.932559,0.315,15588.212559,15588.807559,0,1,13,...,1,1,8,0,"[[132, 56]]","[[[-200]], [[200]], [[-155]], [[155]], [[20]]]","[132, 56]",13,"[[-64.0, -47.0, 113.0, 7.0], [-64.0, -39.0, 40...","[(7990376, 2976, Eye pynwb.behavior.SpatialSer..."
2632,15591.780559,15594.615592,15592.688559,15593.202559,0.360,15593.532559,NaN,0,0,13,...,0,1,8,0,"[[132, 56]]","[[[-200]], [[200]], [[-155]], [[155]], [[20]]]","[132, 56]",13,"[[-64.0, -47.0, 113.0, 7.0], [-64.0, -39.0, 40...","[(7993352, 2831, Eye pynwb.behavior.SpatialSer..."
2633,15599.680592,15602.685592,15600.425592,15600.488592,0.365,15600.822592,15601.287592,0,1,29,...,1,1,9,0,"[[2, 82]]","[[[-200]], [[200]], [[-155]], [[155]], [[20]]]","[2, 82]",13,"[[-9.0, 52.0, 8.0, 43.0], [-50.0, 91.0, 64.0, ...","[(7996183, 2306, Eye pynwb.behavior.SpatialSer..."


In [6]:
trials_df.columns

Index(['start_time', 'stop_time', 'target_presentation_time', 'go_cue_time',
       'reaction_time', 'move_begins_time', 'move_ends_time', 'discard_trial',
       'task_success', 'trial_type', 'trial_version', 'proto_trial',
       'maze_condition', 'correct_reach', 'maze_num_targets',
       'maze_num_barriers', 'novel_maze', 'target_positions', 'frame_details',
       'hit_target_position', 'target_size', 'barrier_info', 'timeseries'],
      dtype='object')

In [9]:
trials_df['timeseries']

id
0       [(0, 3226, Eye pynwb.behavior.SpatialSeries at...
1       [(3226, 2081, Eye pynwb.behavior.SpatialSeries...
2       [(5307, 2846, Eye pynwb.behavior.SpatialSeries...
3       [(8153, 2566, Eye pynwb.behavior.SpatialSeries...
4       [(10719, 2741, Eye pynwb.behavior.SpatialSerie...
                              ...                        
2631    [(7990376, 2976, Eye pynwb.behavior.SpatialSer...
2632    [(7993352, 2831, Eye pynwb.behavior.SpatialSer...
2633    [(7996183, 2306, Eye pynwb.behavior.SpatialSer...
2634    [(7998489, 3631, Eye pynwb.behavior.SpatialSer...
2635    [(8002120, 3201, Eye pynwb.behavior.SpatialSer...
Name: timeseries, Length: 2636, dtype: object

In [11]:
trials_df['timeseries'][0][0][1]

np.int32(3226)

In [13]:
units = nwb.units.to_dataframe()
units

,spike_times,obs_intervals,electrodes,electrode_group
id,,,,
0,"[4.863466666666667, 4.8950000000000005, 4.934,...","[[4.7072, 8.567133333333334], [9.4072000000000...",x y z imp location ...,1 pynwb.ecephys.ElectrodeGroup at 0x1145165838...
1,"[4259.096533333333, 4259.1847333333335, 4259.2...","[[4.7072, 8.567133333333334], [9.4072000000000...",x y z imp location ...,1 pynwb.ecephys.ElectrodeGroup at 0x1145165838...
2,"[4.7085333333333335, 4.802266666666667, 4.8278...","[[4.7072, 8.567133333333334], [9.4072000000000...",x y z imp location ...,1 pynwb.ecephys.ElectrodeGroup at 0x1145165838...
3,"[4.737566666666667, 4.906233333333334, 5.01080...","[[4.7072, 8.567133333333334], [9.4072000000000...",x y z imp location ...,1 pynwb.ecephys.ElectrodeGroup at 0x1145165838...
4,"[4.728400000000001, 4.745233333333333, 4.82703...","[[4.7072, 8.567133333333334], [9.4072000000000...",x y z imp location ...,1 pynwb.ecephys.ElectrodeGroup at 0x1145165838...
...,...,...,...,...
187,"[4.864733333333334, 4.872066666666667, 4.927, ...","[[4.7072, 8.567133333333334], [9.4072000000000...",x y z imp location filterin...,2 pynwb.ecephys.ElectrodeGroup at 0x1144470369...
188,"[4.741066666666667, 4.761166666666667, 4.79336...","[[4.7072, 8.567133333333334], [9.4072000000000...",x y z imp location filterin...,2 pynwb.ecephys.ElectrodeGroup at 0x1144470369...
189,"[4.8323, 4.8963, 4.907, 4.9153, 4.958, 5.07299...","[[4.7072, 8.567133333333334], [9.4072000000000...",x y z imp location filterin...,2 pynwb.ecephys.ElectrodeGroup at 0x1144470369...


In [14]:
spike_ts = units['spike_times']

In [17]:
spike_ts

id
0      [4.863466666666667, 4.8950000000000005, 4.934,...
1      [4259.096533333333, 4259.1847333333335, 4259.2...
2      [4.7085333333333335, 4.802266666666667, 4.8278...
3      [4.737566666666667, 4.906233333333334, 5.01080...
4      [4.728400000000001, 4.745233333333333, 4.82703...
                             ...                        
187    [4.864733333333334, 4.872066666666667, 4.927, ...
188    [4.741066666666667, 4.761166666666667, 4.79336...
189    [4.8323, 4.8963, 4.907, 4.9153, 4.958, 5.07299...
190    [4.8427, 4.8841, 4.903533333333333, 5.05126666...
191    [4.7156, 4.769633333333333, 4.842433333333333,...
Name: spike_times, Length: 192, dtype: object

In [19]:
def get_time_bin(spike_times, check_time, bin_size=1, start_time=0):
    # Convert spike times from seconds to milliseconds
    spike_times_ms = spike_times * 1000
    
    # Define the range for binning
    end_time = np.ceil(np.max(spike_times_ms))
    
    # Create bin edges
    bin_edges = np.arange(start_time, end_time + bin_size, bin_size)
    
    # Check which bin the given time falls into
    bin_index = np.digitize(check_time * 1000, bin_edges) - 1  # Convert to milliseconds and find the bin index

    # Ensure the index is within bounds
    if 0 <= bin_index < len(bin_edges) - 1:
        return bin_index, bin_edges[bin_index], bin_edges[bin_index + 1]
    else:
        return None  # Time is out of range

    
spikes = []
bin_size = 1 #ms
for i, st in enumerate(spike_ts):

    # Create bin edges
    bin_edges = np.arange(int(np.ceil(np.max(st))) + bin_size, bin_size)
    # Bin the spike times and count the spikes in each bin
    binned_sc, _ = np.histogram(st, bins=bin_edges)
    
    spikes.append(binned_sc)

In [49]:
bin_size

1

In [43]:
np.max(st)

np.float64(12417.015364781266)

In [51]:
int(np.ceil(np.max(st))) + bin_size

12419

In [45]:
np.ceil(np.max(st))

np.float64(12418.0)

In [53]:
bin_edges

array([], dtype=int64)

In [31]:
st

array([8.89740000e+00, 8.93616667e+00, 8.97396667e+00, ...,
       1.24169843e+04, 1.24169898e+04, 1.24170154e+04])

In [29]:
spikes

[array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),


In [81]:
trials_df[trials_df['proto_trial'] == 0]

,start_time,stop_time,target_presentation_time,go_cue_time,reaction_time,move_begins_time,move_ends_time,discard_trial,task_success,trial_type,...,correct_reach,maze_num_targets,maze_num_barriers,novel_maze,target_positions,frame_details,hit_target_position,target_size,barrier_info,timeseries
id,,,,,,,,,,,,,,,,,,,,,
0,4.707200,8.567133,5.570200,6.517200,0.335,6.819200,7.234200,0,1,0,...,0,1,2,1,"[[-130, -57]]","[[[-200]], [[200]], [[-155]], [[155]], [[20]]]","[-130, -57]",13,"[[-16.0, 96.0, 59.0, 22.0], [-62.0, -112.0, 43...","[(0, 3226, Eye pynwb.behavior.SpatialSeries at..."
1,9.407200,12.112167,10.172200,10.202200,0.305,10.479200,10.789200,0,1,0,...,0,1,2,1,"[[120, 70]]","[[[-200]], [[200]], [[-155]], [[155]], [[20]]]","[120, 70]",13,"[[-49.0, -59.0, 22.0, 53.0], [-33.0, -100.0, 5...","[(3226, 2081, Eye pynwb.behavior.SpatialSeries..."
2,18.437233,21.972267,19.209233,19.857233,0.300,20.124233,20.584233,0,1,17,...,1,1,9,0,"[[-77, 82]]","[[[-200]], [[200]], [[-155]], [[155]], [[20]]]","[-77, 82]",13,"[[-33.0, 47.0, 6.0, 37.0], [-77.0, 48.0, 11.0,...","[(5307, 2846, Eye pynwb.behavior.SpatialSeries..."
4,26.937267,30.317267,27.630267,28.211267,0.300,28.479267,28.979267,0,1,29,...,1,1,9,0,"[[2, 82]]","[[[-200]], [[200]], [[-155]], [[155]], [[20]]]","[2, 82]",13,"[[-9.0, 52.0, 8.0, 43.0], [-50.0, 91.0, 64.0, ...","[(10719, 2741, Eye pynwb.behavior.SpatialSerie..."
5,31.207233,35.382767,32.149233,33.146233,0.385,33.489233,33.989233,0,1,16,...,1,1,9,0,"[[-118, -83]]","[[[-200]], [[200]], [[-155]], [[155]], [[20]]]","[-118, -83]",13,"[[-33.0, 47.0, 6.0, 37.0], [-77.0, 48.0, 11.0,...","[(13460, 3481, Eye pynwb.behavior.SpatialSerie..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2631,15586.530559,15590.185559,15587.318559,15587.932559,0.315,15588.212559,15588.807559,0,1,13,...,1,1,8,0,"[[132, 56]]","[[[-200]], [[200]], [[-155]], [[155]], [[20]]]","[132, 56]",13,"[[-64.0, -47.0, 113.0, 7.0], [-64.0, -39.0, 40...","[(7990376, 2976, Eye pynwb.behavior.SpatialSer..."
2632,15591.780559,15594.615592,15592.688559,15593.202559,0.360,15593.532559,NaN,0,0,13,...,0,1,8,0,"[[132, 56]]","[[[-200]], [[200]], [[-155]], [[155]], [[20]]]","[132, 56]",13,"[[-64.0, -47.0, 113.0, 7.0], [-64.0, -39.0, 40...","[(7993352, 2831, Eye pynwb.behavior.SpatialSer..."
2633,15599.680592,15602.685592,15600.425592,15600.488592,0.365,15600.822592,15601.287592,0,1,29,...,1,1,9,0,"[[2, 82]]","[[[-200]], [[200]], [[-155]], [[155]], [[20]]]","[2, 82]",13,"[[-9.0, 52.0, 8.0, 43.0], [-50.0, 91.0, 64.0, ...","[(7996183, 2306, Eye pynwb.behavior.SpatialSer..."


In [68]:
trials_df

,start_time,stop_time,target_presentation_time,go_cue_time,reaction_time,move_begins_time,move_ends_time,discard_trial,task_success,trial_type,...,correct_reach,maze_num_targets,maze_num_barriers,novel_maze,target_positions,frame_details,hit_target_position,target_size,barrier_info,timeseries
id,,,,,,,,,,,,,,,,,,,,,
0,4.707200,8.567133,5.570200,6.517200,0.335,6.819200,7.234200,0,1,0,...,0,1,2,1,"[[-130, -57]]","[[[-200]], [[200]], [[-155]], [[155]], [[20]]]","[-130, -57]",13,"[[-16.0, 96.0, 59.0, 22.0], [-62.0, -112.0, 43...","[(0, 3226, Eye pynwb.behavior.SpatialSeries at..."
1,9.407200,12.112167,10.172200,10.202200,0.305,10.479200,10.789200,0,1,0,...,0,1,2,1,"[[120, 70]]","[[[-200]], [[200]], [[-155]], [[155]], [[20]]]","[120, 70]",13,"[[-49.0, -59.0, 22.0, 53.0], [-33.0, -100.0, 5...","[(3226, 2081, Eye pynwb.behavior.SpatialSeries..."
2,18.437233,21.972267,19.209233,19.857233,0.300,20.124233,20.584233,0,1,17,...,1,1,9,0,"[[-77, 82]]","[[[-200]], [[200]], [[-155]], [[155]], [[20]]]","[-77, 82]",13,"[[-33.0, 47.0, 6.0, 37.0], [-77.0, 48.0, 11.0,...","[(5307, 2846, Eye pynwb.behavior.SpatialSeries..."
3,22.777233,26.007267,23.495233,23.959233,0.305,24.229233,24.644233,0,1,8,...,1,1,9,0,"[[132, 56]]","[[[-200]], [[200]], [[-155]], [[155]], [[20]]]","[132, 56]",13,"[[-82.0, -43.0, 8.0, 118.0], [145.0, -105.0, 1...","[(8153, 2566, Eye pynwb.behavior.SpatialSeries..."
4,26.937267,30.317267,27.630267,28.211267,0.300,28.479267,28.979267,0,1,29,...,1,1,9,0,"[[2, 82]]","[[[-200]], [[200]], [[-155]], [[155]], [[20]]]","[2, 82]",13,"[[-9.0, 52.0, 8.0, 43.0], [-50.0, 91.0, 64.0, ...","[(10719, 2741, Eye pynwb.behavior.SpatialSerie..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2631,15586.530559,15590.185559,15587.318559,15587.932559,0.315,15588.212559,15588.807559,0,1,13,...,1,1,8,0,"[[132, 56]]","[[[-200]], [[200]], [[-155]], [[155]], [[20]]]","[132, 56]",13,"[[-64.0, -47.0, 113.0, 7.0], [-64.0, -39.0, 40...","[(7990376, 2976, Eye pynwb.behavior.SpatialSer..."
2632,15591.780559,15594.615592,15592.688559,15593.202559,0.360,15593.532559,NaN,0,0,13,...,0,1,8,0,"[[132, 56]]","[[[-200]], [[200]], [[-155]], [[155]], [[20]]]","[132, 56]",13,"[[-64.0, -47.0, 113.0, 7.0], [-64.0, -39.0, 40...","[(7993352, 2831, Eye pynwb.behavior.SpatialSer..."
2633,15599.680592,15602.685592,15600.425592,15600.488592,0.365,15600.822592,15601.287592,0,1,29,...,1,1,9,0,"[[2, 82]]","[[[-200]], [[200]], [[-155]], [[155]], [[20]]]","[2, 82]",13,"[[-9.0, 52.0, 8.0, 43.0], [-50.0, 91.0, 64.0, ...","[(7996183, 2306, Eye pynwb.behavior.SpatialSer..."


In [70]:
trials_df['correct_reach'].value_counts()

correct_reach
1    2296
0     340
Name: count, dtype: int64

In [21]:
trials_df['target_size'].value_counts()

target_size
13    2636
Name: count, dtype: int64

In [77]:
trials_df.groupby(['correct_reach', 'task_success']).size().reset_index(name='count')

,correct_reach,task_success,count
0,0,0,66
1,0,1,274
2,1,1,2296


In [ ]:
trials_df[trials_df['task_success'] == 0]

In [ ]:
trials_df[trials_df['correct_reach'] == 0]

In [ ]:
trials_df[trials_df.isna().any(axis=1)]

In [ ]:
trial_data

In [ ]:
trial_info[trial_info['success'] == False]

In [ ]:
np.isnan(trial_data['spikes'].to_numpy().flatten()).any()

In [ ]:
trial_lens = []

for trial_id, trial in trial_data.groupby('trial_id'):
    trial_id_trial_info = trial_info[trial_info['trial_id'] == trial_id]
    
    trial_lens.append((trial.trial_time.values[-1] / np.timedelta64(1, 's')) - (trial.trial_time.values[0] / np.timedelta64(1, 's')))
    
plt.hist(trial_lens, bins='auto', density=False, alpha=0.7, edgecolor='gray')

plt.title('trial length\n')
plt.xlabel('time interval of trial length (sec)')
plt.savefig('output_figs/tl.png')
plt.show

In [ ]:
trials = [trial[1] for trial in trial_data.groupby('trial_id')]
inter_trial_intervals = []
    
for i, trial in enumerate(trials):
    trial_id = i + n_null_trials - 1
    trial_id_next = i + 1 + n_null_trials - 1
    trial_id_trial_info = trial_info[trial_info['trial_id'] == trial_id]
    trial_id_trial_info_next = trial_info[trial_info['trial_id'] == trial_id_next]
    
    inter_trial_intervals.append((trial_id_trial_info_next['start_time'].iloc[0] / np.timedelta64(1, 's')) - (trial_id_trial_info['start_time'].iloc[0] / np.timedelta64(1, 's')))
    
plt.hist(inter_trial_intervals, bins='auto', density=False, alpha=0.7, edgecolor='gray')

plt.title('inter-trial time interval\n')
plt.xlabel('inter-trial time interval (sec)')
plt.savefig('output_figs/iit.png')
plt.show()

In [ ]:
time_utill_move = []

for i, _ in enumerate(trials):
    trial_id = i + n_null_trials
    trial_id_trial_info = trial_info[trial_info['trial_id'] == trial_id]
    
    time_utill_move.append((trial_id_trial_info['move_onset_time'].iloc[0] / np.timedelta64(1, 's')) - (trial_id_trial_info['start_time'].iloc[0] / np.timedelta64(1, 's')))
    
plt.hist(time_utill_move, bins='auto', density=False, alpha=0.7, edgecolor='gray')
plt.title('time untill movement\n')
plt.xlabel('time interval before move_onset (sec)')
plt.savefig('output_figs/tibm.png')
plt.show()

In [ ]:
time_after_move = []

for i, _ in enumerate(trials):
    trial_id = i + n_null_trials
    trial_id_trial_info = trial_info[trial_info['trial_id'] == trial_id]
    
    time_after_move.append((trial_id_trial_info['end_time'].iloc[0] / np.timedelta64(1, 's')) - (trial_id_trial_info['move_onset_time'].iloc[0] / np.timedelta64(1, 's')))
    
plt.hist(time_after_move, bins='auto', density=False, alpha=0.7, edgecolor='gray')
plt.title('time after movement\n')
plt.xlabel('time interval after move_onset (sec)')
plt.savefig('output_figs/tiam.png')
plt.show()

### Forming conditions

In [ ]:
def get_simple_cond(angle):
        
    if 350 <= angle < 360 or 0 <= angle < 38:
        return 0
    elif 38 <= angle < 125:
        return 1
    elif 125 <= angle < 175:
        return 2
    elif 175 <= angle < 212:
        return 3
    elif 212 <= angle < 232:
        return 4
    elif 232 <= angle < 280:
        return 5
    elif 280 <= angle <= 329:
        return 6
    elif 329 <= angle <= 350:
        return 7
    else:
        raise ValueError("Angle out of range")

def n_unigue_conds(trial_conds):
    # Convert each list to a frozenset and use a set to track unique frozensets
    unique_conds = set(frozenset(cond) for cond in trial_conds)
    return len(unique_conds)

In [ ]:
# Unique trilas conditions as unique combinations of targets and barriers positions.

trial_conds = []

for trial_id, trial in trial_data.groupby('trial_id'):
    trial_id_trial_info = trial_info[trial_info['trial_id'] == trial_id]
    
    poses = []
    for t_pos in trial_id_trial_info['target_pos']:
        for i in t_pos:
            for t_c in i:
                poses.append(t_c)
            
    for b_pos in trial_id_trial_info['barrier_pos']:
        for j in b_pos:
            for b_c in j:
                poses.append(b_c) 
    
    trial_conds.append(poses)

In [ ]:
## Plot trial-averaged reaches

# Find unique conditions
conds = trial_info.set_index(['trial_type', 'trial_version']).index.unique().tolist()
conds = [cond for cond in conds if not any(math.isnan(x) for x in cond)]

orig_conds = {}
simp_conds = {0:[], 1:[], 2:[], 3:[], 4:[], 5:[], 6:[], 7:[]}

# Initialize plot
fig = plt.figure(figsize=(6, 6))
fig.suptitle('Original trials (varying length)')
ax = fig.add_axes([0.1, 0.1, 0.8, 0.8])

# Loop over conditions and compute average trajectory
for cond_idx, cond in enumerate(conds):
    # Find trials in condition
    mask = np.all(trial_info[['trial_type', 'trial_version']] == cond, axis=1)
    # Extract trial data
    trial_data = dataset.make_trial_data(ignored_trials=(~mask))
    # Average hand position across trials
    traj = trial_data.groupby('align_time')[[('hand_pos', 'x'), ('hand_pos', 'y')]].mean().to_numpy()
    # Determine reach angle for color
    active_target = dataset.trial_info[mask].target_pos.iloc[0][int(dataset.trial_info[mask].active_target.iloc[0])]
    reach_angle = np.arctan2(*active_target[::-1])
    
    orig_conds[cond_idx] = trial_data.trial_id.drop_duplicates().values
    simp_conds[get_simple_cond(math.degrees(reach_angle) + 360 / 2)].append(trial_data.trial_id.drop_duplicates().values)
    
    # Plot reach
    ax.plot(traj[:, 0], traj[:, 1], linewidth=0.7, color=plt.cm.hsv(reach_angle / (2*np.pi) + 0.5))

simp_conds = {key: np.concatenate(value) for key, value in simp_conds.items()}

plt.axis('off')
plt.show()

In [ ]:
## Plot trial-averaged reaches

# Find unique conditions
conds = trial_info.set_index(['trial_type', 'trial_version']).index.unique().tolist()
conds = [cond for cond in conds if not any(math.isnan(x) for x in cond)]

# Initialize plot
fig = plt.figure(figsize=(6, 6))
ax = fig.add_axes([0.1, 0.1, 0.8, 0.8])

# Loop over conditions and compute average trajectory
for cond in conds:
    # Find trials in condition
    mask = np.all(trial_info[['trial_type', 'trial_version']] == cond, axis=1)
    # Extract trial data
    trial_d = dataset.make_trial_data(ignored_trials=(~mask))
    # Average hand position across trials
    traj = trial_d.groupby('align_time')[[('hand_pos', 'x'), ('hand_pos', 'y')]].mean().to_numpy()
    # Determine reach angle for color
    active_target = trial_info[mask].target_pos.iloc[0][int(dataset.trial_info[mask].active_target.iloc[0])]
    reach_angle = np.arctan2(*active_target[::-1])
    # Plot reach
    ax.plot([0, traj[:, 0][-1]], [0, traj[:, 1][-1]], linewidth=0.7, color=plt.cm.hsv(reach_angle / (2*np.pi) + 0.5))

angle_radians = np.radians(350)
x_end = 140 * np.cos(angle_radians)
y_end = 140 * np.sin(angle_radians)
ax.plot([0, x_end], [0, y_end], linewidth=0.7, color='black')
'''
angle_radians = np.radians(16)
x_end = 140 * np.cos(angle_radians)
y_end = 140 * np.sin(angle_radians)
ax.plot([0, x_end], [0, y_end], linewidth=0.7, color='black')
'''
angle_radians = np.radians(38)
x_end = 80 * np.cos(angle_radians)
y_end = 80 * np.sin(angle_radians)
ax.plot([0, x_end], [0, y_end], linewidth=0.7, color='black')

angle_radians = np.radians(125)
x_end = 50 * np.cos(angle_radians)
y_end = 50 * np.sin(angle_radians)
ax.plot([0, x_end], [0, y_end], linewidth=0.7, color='black')

angle_radians = np.radians(175)
x_end = 140 * np.cos(angle_radians)
y_end = 140 * np.sin(angle_radians)
ax.plot([0, x_end], [0, y_end], linewidth=0.7, color='black')
'''
angle_radians = np.radians(193)
x_end = 140 * np.cos(angle_radians)
y_end = 140 * np.sin(angle_radians)
ax.plot([0, x_end], [0, y_end], linewidth=0.7, color='black')
'''
angle_radians = np.radians(212)
x_end = 140 * np.cos(angle_radians)
y_end = 140 * np.sin(angle_radians)
ax.plot([0, x_end], [0, y_end], linewidth=0.7, color='black')

angle_radians = np.radians(232)
x_end = 140 * np.cos(angle_radians)
y_end = 140 * np.sin(angle_radians)
ax.plot([0, x_end], [0, y_end], linewidth=0.7, color='black')

angle_radians = np.radians(280)
x_end = 100 * np.cos(angle_radians)
y_end = 100 * np.sin(angle_radians)
ax.plot([0, x_end], [0, y_end], linewidth=0.7, color='black')

angle_radians = np.radians(329)
x_end = 140 * np.cos(angle_radians)
y_end = 140 * np.sin(angle_radians)
ax.plot([0, x_end], [0, y_end], linewidth=0.7, color='black')

angle_radians = np.radians(0)
x_end = 140 * np.cos(angle_radians)
y_end = 140 * np.sin(angle_radians)
ax.plot([0, x_end], [0, y_end], linewidth=1.7, color='navy')

angle_radians = np.radians(90)
x_end = 50 * np.cos(angle_radians)
y_end = 50 * np.sin(angle_radians)
ax.plot([0, x_end], [0, y_end], linewidth=1.7, color='navy')

angle_radians = np.radians(180)
x_end = 140 * np.cos(angle_radians)
y_end = 140 * np.sin(angle_radians)
ax.plot([0, x_end], [0, y_end], linewidth=1.7, color='navy')

angle_radians = np.radians(270)
x_end = 100 * np.cos(angle_radians)
y_end = 100 * np.sin(angle_radians)
ax.plot([0, x_end], [0, y_end], linewidth=1.7, color='navy')

plt.axis('off')
plt.show()

In [ ]:
fig = plt.figure(figsize=(6, 6))
fig.suptitle('Aligned trials (same length)')
ax = fig.add_axes([0.1, 0.1, 0.8, 0.8])

# Loop over conditions and compute average trajectory
for cod_idx, cond in enumerate([cond for cond in trial_info.set_index(['trial_type', 'trial_version']).index.unique().tolist() if not any(math.isnan(x) for x in cond)]):
    # Find trials in condition
    mask = np.all(dataset.trial_info[['trial_type', 'trial_version']] == cond, axis=1)
    traj = dataset.make_trial_data(align_field='move_onset_time', align_range=(-240, 660), ignored_trials=(~mask)).groupby('align_time')[[('hand_pos', 'x'), ('hand_pos', 'y')]].mean().to_numpy()
    # Determine reach angle for color
    reach_angle = np.arctan2(*trial_info[mask].target_pos.iloc[0][int(trial_info[mask].active_target.iloc[0])][::-1])
    # Plot reach
    ax.plot(traj[:, 0], traj[:, 1], linewidth=0.7, color=plt.cm.hsv(reach_angle / (2*np.pi) + 0.5))

plt.axis('off')
plt.show()

## Forming trials and label vectors

In [ ]:
#[col for col in trial_data.columns if any(_ in col for _ in ['vel', 'pos', 'force', 'acc', 'target'])]
label_cols = [col for col in trial_data.columns if any(_ in col for _ in ['x', 'y'])]

In [ ]:
label_cols

In [ ]:
# Align the trials arount the move_onset bin with offsets before and after that bin.
y = []
labels = []
conds = []

# We want total trial length of 900ms, which is 90 time bins.
bins_before_move = 48
bins_after_move = 132

trial_length = bins_before_move + bins_after_move
n_trials = trial_data.shape[0] // trial_length

for trial_id, trial in trial_data.groupby('trial_id'):
    trial_id_trial_info = trial_info[trial_info['trial_id'] == trial_id]
    
    for cond, trial_ids in orig_conds.items():
        if trial_id in trial_ids:
            conds.append(cond)
            break
            
    # Get the untill movement in ms.
    move_time = (trial_id_trial_info['move_onset_time'].iloc[0] / np.timedelta64(1, 'ms')) - (trial_id_trial_info['start_time'].iloc[0] / np.timedelta64(1, 'ms'))
    # Get the number of bins until movement.
    move_bin = int(move_time // binsize)

    y_heldin_t = torch.tensor(trial.spikes.values)
    y_heldout_t = torch.tensor(trial.heldout_spikes.values)
    
    # Crop the trials arount the move_onset bin with offsets before and after that bin.
    y_t = torch.concat(
        [y_heldin_t[move_bin-bins_before_move:move_bin+bins_after_move, :], y_heldout_t[move_bin-bins_before_move:move_bin+bins_after_move, :]], dim=-1
    )
    
    y.append(y_t.reshape(1, trial_length, n_neurons))
    labels.append(torch.tensor(trial.hand_vel.values[move_bin-bins_before_move:move_bin+bins_after_move, :]).reshape(1, trial_length, 2))
    
y = torch.concat(y, dim=0)
labels = torch.concat(labels, dim=0)
conds = torch.tensor(conds)

print(y.shape)
print(labels.shape)
print(conds.shape)

In [ ]:
conds = []
for trial_id, trial in trial_data.groupby('trial_id'):
    trial_id_trial_info = trial_info[trial_info['trial_id'] == trial_id]
    
    for cond, trial_ids in orig_conds.items():
        if trial_id in trial_ids:
            conds.append(cond)
            break
            
conds = torch.tensor(conds)
print(conds.shape)

In [ ]:
conds

In [ ]:
trial_data

### Forming events occurance time bins

In [ ]:
for i, _ in enumerate(trials):
    trial_id = i + n_null_trials
    trial_id_trial_info = trial_info[trial_info['trial_id'] == trial_id]
    
    target_on = (((trial_id_trial_info['target_on_time'].iloc[0] / np.timedelta64(1, 'ms')) - (trial_id_trial_info['start_time'].iloc[0] / np.timedelta64(1, 'ms'))) // binsize)
    gocue = (((trial_id_trial_info['go_cue_time'].iloc[0] / np.timedelta64(1, 'ms')) - (trial_id_trial_info['start_time'].iloc[0] / np.timedelta64(1, 'ms'))) // binsize)
    move_onset = (((trial_id_trial_info['move_onset_time'].iloc[0] / np.timedelta64(1, 'ms')) - (trial_id_trial_info['start_time'].iloc[0] / np.timedelta64(1, 'ms'))) // binsize)

In [ ]:
target_bins = []
gocue_bins = []
move_bins = []
event_bins = []

for i, _ in enumerate(trials):
    trial_id = i + n_null_trials
    trial_id_trial_info = trial_info[trial_info['trial_id'] == trial_id]
    
    # target : go
    delay = (((trial_id_trial_info['go_cue_time'].iloc[0] / np.timedelta64(1, 'ms')) - (trial_id_trial_info['target_on_time'].iloc[0] / np.timedelta64(1, 'ms'))) // binsize)
    # go : move
    prep = (((trial_id_trial_info['move_onset_time'].iloc[0] / np.timedelta64(1, 'ms')) - (trial_id_trial_info['go_cue_time'].iloc[0] / np.timedelta64(1, 'ms'))) // binsize)
    
    target_on = (((trial_id_trial_info['target_on_time'].iloc[0] / np.timedelta64(1, 'ms')) - (trial_id_trial_info['start_time'].iloc[0] / np.timedelta64(1, 'ms'))) // binsize)
    gocue = (((trial_id_trial_info['go_cue_time'].iloc[0] / np.timedelta64(1, 'ms')) - (trial_id_trial_info['start_time'].iloc[0] / np.timedelta64(1, 'ms'))) // binsize)
    move_onset = (((trial_id_trial_info['move_onset_time'].iloc[0] / np.timedelta64(1, 'ms')) - (trial_id_trial_info['start_time'].iloc[0] / np.timedelta64(1, 'ms'))) // binsize)
    
    target_bins.append(target_on - move_onset + bins_before_move)
    gocue_bins.append(gocue - move_onset + bins_before_move)
    move_bins.append(bins_before_move)

event_bins.append(torch.tensor(target_bins))
event_bins.append(torch.tensor(gocue_bins))
event_bins.append(torch.tensor(move_bins))
event_bins = torch.stack(event_bins)
event_bins.shape

In [ ]:
event_bins = event_bins.permute(1, 0)

event_bins[event_bins < 0] = float('nan')
event_bins[event_bins > bins_before_move + bins_after_move] = float('nan')
event_bins.shape

In [ ]:
event_bins

### Save data splits

In [ ]:
import torch

!mkdir data
save_root_path = 'data/'

train_data, valid_data, test_data = {}, {}, {}
n_trials, seq_len, n_neurons = y.shape
n_valid_trials = 574

# obs: observations
train_data['y_obs'] = torch.Tensor(y[:-n_valid_trials])
valid_data['y_obs'] = torch.Tensor(y[-n_valid_trials:-n_valid_trials // 2])
test_data['y_obs'] = torch.Tensor(y[-n_valid_trials // 2:])

# 'n_bins_enc': Number of time bins used later by in modeling for enconding (default full trial).
# 'n_bins_obs': originaly observed trial length (after alignment)
# Same for 'n_neurons_obs' and 'n_neurons_enc'.
train_data['n_bins_obs'] = valid_data['n_bins_obs'] = test_data['n_bins_obs'] = seq_len
train_data['n_bins_enc'] = valid_data['n_bins_enc'] = test_data['n_bins_enc'] = seq_len
train_data['n_neurons_obs'] = valid_data['n_neurons_obs'] = test_data['n_neurons_obs'] = n_neurons
train_data['n_neurons_enc'] = valid_data['n_neurons_enc'] = test_data['n_neurons_enc'] = n_neurons

# Save a 1D array for event bins for each data split, for each trial, for each event.
# Note: the o here in event_bins[0] is the session-animal group.
for event_id, event in enumerate(['targrt_on_bin', 'go_cue_bin', 'move_onset_bin']):
    train_data[event] = torch.Tensor(np.array(event_bins[:-n_valid_trials, event_id]))
    valid_data[event] = torch.Tensor(np.array(event_bins[-n_valid_trials:-n_valid_trials // 2, event_id]))
    test_data[event] = torch.Tensor(np.array(event_bins[-n_valid_trials // 2:, event_id]))

train_data['hand_vel'] = torch.Tensor(np.array(labels[:-n_valid_trials, :, :]))
valid_data['hand_vel'] = torch.Tensor(np.array(labels[-n_valid_trials:-n_valid_trials // 2, :, :]))
test_data['hand_vel'] = torch.Tensor(np.array(labels[-n_valid_trials // 2:, :, :]))
    
'''
for label_id, label in enumerate(label_cols):
    train_data[f'{label[0]}_{label[1]}'] = torch.Tensor(np.array(labels[:-n_valid_trials, :, :]))
    valid_data[f'{label[0]}_{label[1]}'] = torch.Tensor(np.array(labels[-n_valid_trials:-n_valid_trials // 2, :, :]))
    test_data[f'{label[0]}_{label[1]}'] = torch.Tensor(np.array(labels[-n_valid_trials // 2:, :, :]))
'''
torch.save(train_data, save_root_path + f'data_train_{binsize}ms.pt')
torch.save(valid_data, save_root_path + f'data_valid_{binsize}ms.pt')
torch.save(test_data, save_root_path + f'data_test_{binsize}ms.pt')

print('Data splits (train/valid/test) saved into the "data" folder.')

## Load data splits

In [ ]:
data_path = 'data/data_{split}_{bin_sz_ms}ms.pt'
train_data = torch.load(data_path.format(split='train', bin_sz_ms=cfg.bin_sz_ms))
val_data = torch.load(data_path.format(split='valid', bin_sz_ms=cfg.bin_sz_ms))
test_data = torch.load(data_path.format(split='test', bin_sz_ms=cfg.bin_sz_ms))

y_train_obs = train_data['y_obs'].type(torch.float32).to(cfg.data_device)
y_valid_obs = val_data['y_obs'].type(torch.float32).to(cfg.data_device)
y_test_obs = test_data['y_obs'].type(torch.float32).to(cfg.data_device)

vel_train = train_data['velocity'].type(torch.float32).to(cfg.data_device)
vel_valid = val_data['velocity'].type(torch.float32).to(cfg.data_device)
vel_test = test_data['velocity'].type(torch.float32).to(cfg.data_device)

y_train_dataset = torch.utils.data.TensorDataset(y_train_obs, vel_train)
y_val_dataset = torch.utils.data.TensorDataset(y_valid_obs, vel_valid)
y_test_dataset = torch.utils.data.TensorDataset(y_test_obs, vel_test)

train_dataloader = torch.utils.data.DataLoader(y_train_dataset, batch_size=cfg.batch_sz, shuffle=True)
valid_dataloader = torch.utils.data.DataLoader(y_val_dataset, batch_size=y_valid_obs.shape[0], shuffle=False)
test_dataloader = torch.utils.data.DataLoader(y_test_dataset, batch_size=y_valid_obs.shape[0], shuffle=False)

# Data dimensions
n_train_trials, n_time_bins, n_neurons_obs = y_train_obs.shape
n_valid_trials = y_valid_obs.shape[0]
n_test_trials = y_test_obs.shape[0]
n_time_bins_enc = train_data['n_time_bins_enc']

batch_sz_train = list(y_train_obs.shape)[:-1]
batch_sz_valid = list(y_valid_obs.shape)[:-1]
batch_sz_test = list(y_test_obs.shape)[:-1]

print("# training trials: {0}".format(n_train_trials))
print("# validation trials: {0}".format(n_valid_trials))
print("# testing trials: {0}".format(n_test_trials))
print("# neurons: {0}".format(n_neurons_obs))
print("# time bins: {0}".format(n_time_bins))
print("# time bins used for forcasting: {0}".format(cfg.n_bins_bhv))
print("# predicted time bins: {0}".format(n_time_bins - cfg.n_bins_bhv))